# Implementación del algóritmo genético en 1D

Para comenzar, se importa de la librería de numpy para poder trabajar con arreglos

In [5]:
import numpy as np

## Definir población, probabilidades, etc.

In [6]:
r_cross = 0.9
r_mut= 0.25

pop=np.array([[0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 1, 1], [0, 0, 0, 1], [0, 1, 0, 1], [0, 0, 1, 0]])

l_sel= [[2, 3, 5], [2, 4, 5], [0, 4, 1], [4, 0, 5], [3, 1, 5], [0, 5, 4], [4, 1, 0], [3, 4, 0],
[1, 4, 2], [5, 4, 3], [0, 1, 5], [0, 2, 1], [2, 1, 3], [2, 5, 3], [3, 5, 1], [3, 0, 1], [4, 3,
1], [1, 5, 4], [1, 5, 2], [4, 2, 3], [1, 5, 4], [0, 1, 2], [5, 2, 1], [5, 0, 4]]

l_cross= [1, 2, 2, 1, 2, 2, 2, 2, 2, 2]

l_rand= [0.70, 0.13, 0.38, 0.29, 0.85, 0.68, 0.53, 0.49, 0.03, 0.06, 0.78, 0.17,
0.08, 0.01, 0.46, 0.14, 0.18, 0.26, 0.56, 0.79, 0.60, 0.43, 0.00, 0.21, 0.21, 0.33,
0.44, 0.91, 0.99, 0.00, 0.97, 0.98, 0.83, 0.11, 0.62, 0.66, 0.33, 0.27, 0.54, 0.20,
0.06, 0.98, 0.27, 0.57, 0.46, 0.97, 0.18, 0.31, 0.19, 0.70, 0.84, 0.99, 0.92, 0.09,
0.36, 0.25, 0.14, 0.88, 0.26, 0.76, 0.90, 0.85, 0.73, 0.28, 0.82, 0.43, 0.17, 0.23,
0.73, 0.95, 0.00, 0.30, 0.04, 0.74, 0.66, 0.34, 0.18, 0.37, 0.52, 0.94, 0.84, 0.75,
0.26, 0.96, 0.01, 0.91, 0.99, 0.21, 0.69, 0.72, 0.48, 0.70, 0.14, 0.53, 0.96, 0.14,
0.33, 0.41, 0.47, 0.67, 0.75, 0.28, 0.90, 0.00, 0.65, 0.97, 0.59, 0.06]

# valor maximo que se puede obtener con la resolución dada de la población
val_max = 2 ** len(pop[0]) - 1

# contador para la seleccion
cont_sel = 0

# contador para los random
cont_rand = 0

# contador para el cruzamiento
cont_cross = 0

## Implementación de funciones

Ahora se implementan funciones necesarias para el algoritmo, como la funcion objetivo, funcion de selección, función para mutación, etc.

In [7]:
# funcion para definir la función objetivo que es |2x-1.2|
def f_obj(x):
    """
    Función objetivo: |2x-1.2|
    input: arreglo de numpy con los valores enteros de la población
    output: fitness de la población
    """
    fitness = np.abs(2 * x - 1.2)
    return fitness

# funcion para convertir binarios a entero
def bin2int(pop):
    """
    Esta funcion convierte los valores binarios de la población a enteros
    input: valores binarios de la población
    output: valores enteros de la población
    """
    # Lista para guardar los valores enteros de la poblacion
    pop_entero = [] 
    # Convertir a entero cada particula de la poblacion
    for particle in pop:
        # variable auxiliar para ir guardando el valor de la suma de cada bit
        sum = 0
        # En el siguiente for cada bit se multiplica por la potencia de dos correspondiente
        for idx, n in enumerate(range(len(particle)-1, -1, -1)):
            sum += particle[idx] * 2 ** n
        # Guardar el valor entero en la lista de enteros
        pop_entero.append(sum)
    return np.array(pop_entero)

# Funcion para mapear el numero entero al valor decodificado
def mapeo(pop_entero, rango: list = [-1, 1]):
    """
    Esta funcion realiza el mapeo de un numero entero a un valor decodificado en el rango dado
    input: lista de los numeros enteros a mapear y lista con el rango en el que se mapearán, el default es de -1 a 1
    output: lista con los valores decodificados
    """
    
    # primero se obtiene la ecuación para mapear los numeros esta es la ecuación de una recta ya que es un mapeo lineal
    m = (rango[1] - rango [0]) / (val_max)
    # mapear los valores
    decoded = m * pop_entero + rango[0]
    return np.array(decoded) 

# Funcion para evaluar el fitness de las particulas
def eval_fitness(decoded):
    """
    Esta funcion evalua el fitness de los valores dados con la funcion objetivo definida
    input: lista con los valores decodificados
    output: lista con la evaluación del fitness en la funcion objetivo
    """
    return f_obj(decoded)

# Funcion para realizar la selección por torneo
def seleccion(pop, fitness_pop):
    """
    Esta funcion realiza la seleccion por torneo
    input: poblacion y el fitness de cada particula
    output: lista con las partitcuas seleccionadas
    """
    global cont_sel
    # lista para guardar a los ganadores
    pop_sel = []
    # Realizar un torneo por cada particula en la poblacion
    for i in range(len(pop)):
        # obtener los fitness de los elegidos para competir
        fitness_torneo = fitness_pop[l_sel[cont_sel]]
        # Obtener el indice de la particula con mejor fitness
        idx_ganador = np.argmin(fitness_torneo)
        # Seleccionar la particula con ese fitness
        ganador = pop[l_sel[cont_sel][idx_ganador]]
        pop_sel.append(ganador)
        # aumentar el contador para obtener el siguiente elemento de la lista de seleccion
        cont_sel +=1
    return np.array(pop_sel)

# Funcion para realizar el cruzamiento y la mutacion
def cross_mut(pop_nueva):
    """
    Esta funciónn realiza el cruzamiento de la población y la mutacion
    input: población en la que se realizara el cruzamiento y mutacion
    output: población con el cruzamiento y la mutacion
    """
    global cont_cross
    global cont_rand
    # Lista para guardar las particlulas con el cruzamiento
    pop_cross = []
    # Tomar particulas de dos en dos para el cruzamiento
    for i in range(0, len(pop_nueva), 2):
        # Hacer el cruzamiento solo si la probabilidad
        if l_rand[cont_rand] <= r_cross:
            # Obtener la posición del cruzamiento
            pos = l_cross[cont_cross]
            # particula 1 con cruzamiento
            part1 = np.concatenate((pop_nueva[i, :pos], pop_nueva[i+1, pos:]))
            # particula 2 con cruzamiento
            part2 = np.concatenate((pop_nueva[i+1, :pos], pop_nueva[i, pos:]))
            # Aumentar el contador para obtener el siguiente elemento de la selección
            cont_cross +=1
        else:  # si no hay cruzamiento las particulas quedan igual
            part1 = pop_nueva[i]
            part2 = pop_nueva[i+1]
        
        # Aumentar el contador de los randoms para la mutacion
        cont_rand += 1
        
        # Ahora se realiza la mutación de las dos particulas obtenidas despues del cruzamiento
        for j in [part1, part2]:
            # Hacer la mutación de cada bit
            for k, bit in enumerate(j):
                # Si está dentro de la probabilidad se muta el bit
                if l_rand[cont_rand] <= r_mut:
                    # Invertir el valor del bit para mutar
                    j[k] = 1 - bit
                # aumentar el contador de randoms
                cont_rand += 1
        
        pop_cross.append(part1)
        pop_cross.append(part2)
        
    return np.array(pop_cross)


## Implementación del algoritmo

Una vez que ya se tienen todas las funciones necesarias ahora se llaman estas funciones dentro de un bucle para entrenar varias generaciones

In [8]:
gen = 0
# tolerancia para detener el algoritmo
tol = 1e-3
# Definimos un mejor fitness grande para iniciar el algoritmo
mejor_fitness= np.infty

# Seguir con el algoritmo mientra sean menos de 100 generaciones o el fitness sea mayor a 0.001
while((mejor_fitness > tol) and (gen < 100)):
    # Primero se convierte la población en binario a entero
    pop_int = bin2int(pop)
    # Luego se mapean esos enteros al rango dado
    pop_decoded = mapeo(pop_int)
    # Se evalua el fitness de la población
    pop_fitness = eval_fitness(pop_decoded)
    # Obtener el mejor fitness de la poblacion
    mejor_fitness_pop = min(pop_fitness)
    # Si el mejor fitness de la poblacion es menor al mejor fitness global se actualiza este
    if mejor_fitness_pop < mejor_fitness:
        mejor_fitness = mejor_fitness_pop
        # Obtener la particula con el mejor fitness
        part_opt = pop[np.argmin(pop_fitness)]
        decoded_opt = mapeo(bin2int([part_opt]))[0]
        fitness_opt = eval_fitness(mapeo(bin2int([part_opt])))[0]
    # Se realiza la seleccion por torneo
    pop_nueva = seleccion(pop, pop_fitness)
    # Se realiza el cruzamiento y mutacion, esta será la nueva población
    pop = cross_mut(pop_nueva)
    gen += 1
    
# Una vez concluido el algoritmo se muestran los resultados
print(f"La mejor particula es {part_opt}, con un valor decodificado de {decoded_opt} y un fitness de {fitness_opt}")
print(f"Obtenido en la generación {gen-1}")

La mejor particula es [1 1 0 0], con un valor decodificado de 0.6000000000000001 y un fitness de 2.220446049250313e-16
Obtenido en la generación 2
